# Introduction
The objective of this notebook is to provide a brief introduction to the SIR compartmental epidemiology model and then
demonstrate how to run the model for various parameters. Christian Hill's SIR epidemic model example in
[Learning Scientific Programming with Python](https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/)
served as inspiration for some of the material presented herein.

# Background
The SIR [model](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology) is an epidemiological model that is useful for understanding how infectious disease in humans and animals evolve over time. It is a compartmental model
that divides the population into susceptible, infectious, and recovered groups. Given an initial population
distribution, it models the evolution of the number of people in each compartment over time.

\begin{equation*}
\large Susceptible\xrightarrow{\beta}Infectious\xrightarrow{\gamma}Recovered
\end{equation*}

where $\beta$ is the transmission rate and $\gamma$ is the recovery rate. Both are rates and therefore have units
expressed in per unit time. Alternatively, $1/\beta$ and $1/\gamma$ are expressed in time units such as days, weeks,
or months. For the purposes of the SIR model, the recovered group no longer affects disease transmission because
they are either recovered, deceased, or immune.

# Why?
When studying computational fluid dynamics in graduate school we sometimes found that our computed fluid flows failed
to capture all of the features observed in the lab. In these cases, we always reminded ourselves that "nature got the
flow right." Models do not always capture every phenomena. The aphorism
["All models are wrong, but some are useful"](https://en.wikipedia.org/wiki/All_models_are_wrong) holds true for the
SIR model. Populations are not constant. Transmission rates are not constant. That said, the model provides insights
into the complex interactions that occur in real transmission. It shows the interaction between
transmission rate and recovery rate and how they interact in ways that are not intuitive.

The SIR model is also important because it serves as the basis for other compartmental models. The simpler SI model can
be derived as a special case when the recovery time is infinite. The SIS model
can be used to model the common cold where recovery does not confer immunity. The SIRS model is an extension where
immunity wanes over time like the flu. Armed with the information below these models will be within reach.

# The SIR Model

The SIR model starts with the assumption that the entire population can be divided into Susceptible, Infectious, and
Recovered groups denoted by $S$, $I$, and $R$ respectively. The total population, denoted by $N$, is assumed to
remain constant. These values are related as follows:

\begin{equation*}
S(t) + I(t) + R(t) = N = {\text{constant}}
\end{equation*}

Our goal is to model how the number of people in each group change over time. Taking the derivative with respect to
time yields:

\begin{equation*}
\frac{dS}{dt} + \frac{dI}{dt} +\frac{dR}{dt} = 0
\end{equation*}

The rate of change must net out to zero. If one bucket is draining another bucket must be filling up. The system of ordinary differential equations governing how the population in each bucket grow and shrinks is given by:

\begin{aligned}
{\frac {dS}{dt}} & = -{\frac {\beta SI}{N}},\\
{\frac {dI}{dt}} & = {\frac {\beta SI}{N}}-\gamma I,\\
{\frac {dR}{dt}} & = \gamma I,
\end{aligned}

It is important to remember that the variables $S$, $I$, and $R$ are always positive and may evolve with time while
$N$, $\beta$, and $\gamma$ are all positive constants. Given an initial population for each compartment, we can evolve
the populations in each bucket over time. Note that $S$ may never grow because $\beta$, $I$, $S$, and $N$ are always
positive which ensures that the rate is negative. Likewise, the population in the recovered compartment, $R$, may never decrease because both $\gamma$ and $I$
are positive ensuring that the growth rate is always postive. By contrast, the infectious population may grow or shrink depending on relative size of the two terms.

We can generalize this model to populations of any size by thinking in terms of
percent susceptible, infected and recovered as follows:

\begin{aligned}
s(t) & = {\frac {S(t)}{N}} \\
i(t) & = {\frac {I(t)}{N}} \\
r(t) & = {\frac {R(t)}{N}} \\
\end{aligned}

Substitution $S=sN$, $I=iN$, and $R=rN$ into the original equations yields:

\begin{equation*}
s(t) + i(t) + r(t) = 1
\end{equation*}

\begin{equation*}
\frac{ds}{dt} + \frac{di}{dt} +\frac{dr}{dt} = 0
\end{equation*}

\begin{aligned}
{\frac {ds}{dt}} & =-\beta si,\\
{\frac {di}{dt}} & = \beta si - \gamma i,\\
{\frac {dr}{dt}} & = \gamma i,
\end{aligned}

Note that the equations above are independent of populations size $N$. 

# The basic reproduction ratio $R_0$

We have already scaled $s$, $i$ and $r$ by the total population size $N$. Our goal is now to non-dimensionalize our dependent time variable. Both the transmission and recovery rates are candidates for scaing time. We will choose $\beta t$ as a dimensionless time scale. Dividing through our previous equations by $\beta$ and introducing $\tau=\beta t$ as our dimensionless time parameter yields

\begin{equation*}
s(\tau) + i(\tau) + r(\tau) = 1
\end{equation*}

\begin{equation*}
\frac{ds}{d\tau} + \frac{di}{d\tau} +\frac{dr}{d\tau} = 0
\end{equation*}

\begin{aligned}
{\frac {ds}{d\tau}} & = -si,\\
{\frac {di}{d\tau}} & = si - \frac{i}{R_0} =i \left( s - \frac{1}{R_0} \right) ,\\
{\frac {dr}{d\tau}} & = \frac{i}{R_0} ,
\end{aligned}

where we have introduced $R_0 = \beta/\gamma $ as the [basic reproduction ratio](https://en.wikipedia.org/wiki/Basic_reproduction_number) which is the ratio of the transmission rate to the recovery rate. $R_0$ can be thought of as the average number of secondary cases arising from an average primary case in an entirely susceptible population. If $R_0$ is greater than 1 it means that the rate of infection may instantaneously outpace the rate of recovery resulting in an outbreak. We now have the SIR model with a single parameter, $R_0$. The constants $N$, $\beta$, and $\gamma$ scale the solution but do not change the evolution of the compartments. Let us now solve these equations to determine the time evolution of the populations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# The SIR model differential equations.
def sir_model(t, y):
    # basic reproduction ratio
    R_0 = 2

    s, i, r = y
    dsdtau = -s * i
    didtau =  s * i - i/R_0
    drdtau =  i/R_0
    return dsdtau, didtau, drdtau


# Initial number of infected and recovered individuals
# Everyone else is susceptible to infection initially.
r_initial = 0
i_initial = 0.001
s_initial = 1 - i_initial - r_initial

# Integrate the SIR equations over the time
# ivp means initial value problem (as opposed to a boundary value problem)
t_initial = 0
t_final = 40
sol = solve_ivp(sir_model, (t_initial, t_final), (s_initial, i_initial, r_initial),
                t_eval=np.linspace(t_initial, t_final, 1000))
s, i, r = sol.y

plt.plot(sol.t, 100*s, label='Susceptible')
plt.plot(sol.t, 100*i, label='Infected')
plt.plot(sol.t, 100*r, label='Recovered')
plt.xlabel(r"$\beta t$")
plt.ylabel('Percent')
plt.xlim((t_initial, t_final))
plt.ylim((0,100))
plt.grid()
plt.legend()
plt.show()

plt.plot(s, i)
plt.xlabel(r"s")
plt.ylabel(r"i")
plt.xlim((0,1))
plt.ylim((0,1))
plt.grid()
plt.show()

# What is solve_ivp actually doing?

The SIR model is an initial value problem. We choose initial values for $s_0$ (close to 1), $i_0$ (close to 0), and $r_0$ (the initial recovered population not to be confused with $R_0$) and want to understand "the next value" for each. We can think of the slope as the difference between successive values of $s$, $i$, and $r$ for short time steps. If we know the all three values at time step $n$ we can approximate the values at time $n+1$ as follows

\begin{aligned}
{\frac {ds}{d\tau}} & \approx {\frac {\Delta s}{\Delta\tau}} = {\frac {s_{n+1} - s_n}{\Delta\tau}} = - s_n i_n \\
{\frac {di}{d\tau}} & \approx {\frac {\Delta i}{\Delta\tau}} = {\frac {i_{n+1} - i_n}{\Delta\tau}} = s_n i_n - i_n /R_0   \\
{\frac {dr}{d\tau}} & \approx {\frac {\Delta r}{\Delta\tau}} = {\frac {r_{n+1} - r_n}{\Delta\tau}} = i_n / R_0 \\
\end{aligned}

where $\Delta\tau$ is known small time step of our choosing. Rearranging and solving for the $n+1$ terms yields

\begin{aligned}
s_{n+1} & = s_n - \Delta\tau s_n i_n   \\
i_{n+1} & = i_n + \Delta\tau (s_n i_n - i_n / R_0)   \\
r_{n+1} & = r_n + \Delta\tau i_n / R_0 \\
\end{aligned}

Note that the difference equations listed above are quite simple to solve using any programming language or Excel. In scientific computing, this technique for estimating the next time step is called the forward Euler method because we chose to evaluate the slope using $s_n$ and $i_n$. There are other choices that we could have made. The Adams–Bashforth method determines the next value using a weighted average of the two previous values. By default, solve_ivp uses the Runge-Kutta method but the concept is the same.

In [ ]:
import sympy as sym

s, s0, s1, i, i0, i1, r, r0, r1, epsilon = sym.symbols('s s0 s1 i i0 i1 r r0 r1 epsilon')
dsdt = - s * i
didt =   s * i - epsilon * i
drdt =           epsilon * i
cont = s + i + r

dsdt_pert = dsdt.subs(s, s0 + epsilon * s1).subs(i, i0 + epsilon * i1)
didt_pert = didt.subs(s, s0 + epsilon * s1).subs(i, i0 + epsilon * i1)
drdt_pert = drdt.subs(s, s0 + epsilon * s1).subs(i, i0 + epsilon * i1)
cont_pert = cont.subs(s, s0 + epsilon * s1).subs(i, i0 + epsilon * i1).subs(r, r0 + epsilon * r1)

print(sym.collect(sym.expand(dsdt_pert.series(epsilon, 0, 2)), epsilon))
print(sym.collect(sym.expand(didt_pert.series(epsilon, 0, 2)), epsilon))
print(sym.collect(sym.expand(drdt_pert.series(epsilon, 0, 2)), epsilon))
print(sym.collect(sym.expand(cont_pert.series(epsilon, 0, 2)), epsilon))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

R_0 = 2

N = 20
s = np.linspace(0, 1, N)
i = s.copy()
S, I = np.meshgrid(s, i)

didt = I*(S - 1/R_0)

fig, ax = plt.subplots()
ax.axis('square')
ax.set(xlim=(0,1), ylim=(0,1))
plt.contour(S, I, I*(S - 1/R_0), levels=np.linspace(-1,1,21), cmap= 'RdBu')
plt.show()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(S, I, I*(S - 1/R_0))
plt.show()